In [3]:
import pandas as pd
import plotly.express as px
import numpy as np
import datetime
import seaborn as sns
import plotly.graph_objects as go
import reverse_geocoder as rg
import swifter

# load the data

In [13]:
df = pd.read_csv('./meteorites_clean.csv')
df

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation
0,Aachen,1,Valid,L5,21.0,Fell,1880,50.77500,6.08333,"(50.775, 6.08333)"
1,Aarhus,2,Valid,H6,720.0,Fell,1951,56.18333,10.23333,"(56.18333, 10.23333)"
2,Abee,6,Valid,EH4,107000.0,Fell,1952,54.21667,-113.00000,"(54.21667, -113.0)"
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976,16.88333,-99.90000,"(16.88333, -99.9)"
4,Achiras,370,Valid,L6,780.0,Fell,1902,-33.16667,-64.95000,"(-33.16667, -64.95)"
...,...,...,...,...,...,...,...,...,...,...
31942,Zillah 002,31356,Valid,Eucrite,172.0,Found,1990,29.03700,17.01850,"(29.037, 17.0185)"
31943,Zinder,30409,Valid,"Pallasite, ungrouped",46.0,Found,1999,13.78333,8.96667,"(13.78333, 8.96667)"
31944,Zlin,30410,Valid,H4,3.3,Found,1939,49.25000,17.66667,"(49.25, 17.66667)"
31945,Zubkovsky,31357,Valid,L6,2167.0,Found,2003,49.78917,41.50460,"(49.78917, 41.5046)"


In [4]:
country_code = pd.read_csv('./data/country_code.csv')
country_code

,Pays,ISO 3166-1 alpha-2,ISO 3166-1 alpha-3,ISO 3166-1 numérique
0,Afghanistan,AF,AFG,4
1,Îles Åland,AX,ALA,248
2,Albanie,AL,ALB,8
3,Algérie,DZ,DZA,12
4,Samoa américaines,AS,ASM,16
...,...,...,...,...
242,Wallis-et-Futuna,WF,WLF,876
243,Sahara occidental,EH,ESH,732
244,Yémen,YE,YEM,887
245,Zambie,ZM,ZMB,894


## Get country from the coordinates
I used reverse geocoder to get the country and other info from the raw coordinates of the meteorites.
Then I merge with another data set to have the country name and the ISO-3 code.

In [59]:
# Aggregated by country
#In this part we continue the analysis by aggregating by country and not coordinates

# df_country = df.copy()
# df_country1 = df_country[(df_country["year"] >= 1999)].copy()
# df_country2 = df_country[(df_country["year"] <= 1986)].copy()
# df_country3 = df_country[(df_country["year"] > 1986) & (df_country["year"] < 1999)].copy()

In [5]:
def get_country_code(row):
    return rg.search((row['reclat'], row['reclong']), mode=1)[0]['cc']

In [11]:
%%time
df_country = pd.DataFrame()
for chunk in np.array_split(df,1000):
    chunk['ISO 3166-1 alpha-2'] = chunk.swifter.progress_bar(False).apply(lambda row: get_country_code(row), axis=1)
    df_country = df_country.append(chunk)
    

# df_country1['ISO 3166-1 alpha-2'] = df_country1.swifter.progress_bar(False).apply(lambda row: get_country_code(row), axis=1)
# df_country2['ISO 3166-1 alpha-2'] = df_country2.swifter.progress_bar(False).apply(lambda row: get_country_code(row), axis=1)
# df_country3['ISO 3166-1 alpha-2'] = df_country3.swifter.progress_bar(False).apply(lambda row: get_country_code(row), axis=1)
# df_country = pd.concat([df_country1, df_country2, df_country3])
# del df_country1, df_country2, df_country3 

Wall time: 18.8 s


In [14]:
df_country.info()
df_country = df_country.merge(country_code, on=['ISO 3166-1 alpha-2'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31947 entries, 0 to 31946
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                31947 non-null  object 
 1   id                  31947 non-null  int64  
 2   nametype            31947 non-null  object 
 3   recclass            31947 non-null  object 
 4   mass (g)            31894 non-null  float64
 5   fall                31947 non-null  object 
 6   year                31947 non-null  int64  
 7   reclat              31947 non-null  float64
 8   reclong             31947 non-null  float64
 9   GeoLocation         31947 non-null  object 
 10  ISO 3166-1 alpha-2  31947 non-null  object 
dtypes: float64(3), int64(2), object(6)
memory usage: 2.7+ MB


In [15]:
df_country.drop(['ISO 3166-1 alpha-2', 'ISO 3166-1 numérique'], axis=1, inplace=True)
df_country

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation,Pays,ISO 3166-1 alpha-3
0,Aachen,1,Valid,L5,21.0,Fell,1880,50.77500,6.08333,"(50.775, 6.08333)",Allemagne,DEU
1,Barntrup,4948,Valid,LL4,17.0,Fell,1886,52.00000,9.10000,"(52.0, 9.1)",Allemagne,DEU
2,Breitscheid,5134,Valid,H5,1500.0,Fell,1956,50.66694,8.18361,"(50.66694, 8.18361)",Allemagne,DEU
3,Bremervörde,5135,Valid,H/L3.9,7250.0,Fell,1855,53.40000,9.10000,"(53.4, 9.1)",Allemagne,DEU
4,Darmstadt,6603,Valid,H5,100.0,Fell,1804,49.86667,8.65000,"(49.86667, 8.65)",Allemagne,DEU
...,...,...,...,...,...,...,...,...,...,...,...,...
31922,Rosario,22765,Valid,"Iron, IAB-MG",2700.0,Found,1896,14.60000,-88.68333,"(14.6, -88.68333)",Honduras,HND
31923,Ryder Gletcher,22790,Valid,L5,5288.0,Found,1988,81.16667,-49.00000,"(81.16667, -49.0)",Groenland,GRL
31924,Sainte Rose,23720,Valid,H3.6,430.0,Found,1983,-21.19222,55.68639,"(-21.19222, 55.68639)",Réunion,REU
31925,Shohaku,23580,Valid,Iron,101.0,Found,1938,40.31667,126.91667,"(40.31667, 126.91667)",Corée du Nord,PRK


In [8]:
#df_country.to_csv (r'./meteorites_country.csv', index = False, header=True)

## Adding two other data sets to add info on the countries (population, size, name in english...)

In [21]:
# df_meteorites = pd.read_csv('./data/meteorites_country.csv')
df_meteorites = df_country
df_countries = pd.read_csv('./data/countries_world.csv')
df_iso = pd.read_csv('./data/iso-codes.csv')
df_countries['Country'] = df_countries['Country'].str[:-1]

In [22]:
df_iso.rename(columns={'Alpha-3 code': 'ISO 3166-1 alpha-3'}, inplace=True)
df_meteorites = df_meteorites.merge(df_iso, on=['ISO 3166-1 alpha-3'])
df_meteorites.drop(['Pays', 'Alpha-2 code', 'GeoLocation', 'Numeric code', 'ISO 3166-2'], axis=1, inplace=True)

In [23]:
#handle names exceptions to merge with other data on country
def rename_country(row):
    if row['ISO 3166-1 alpha-3'] == 'USA':
        return 'United States'
    elif row['ISO 3166-1 alpha-3'] == 'LBY':
        return 'Libya'
    elif row['ISO 3166-1 alpha-3'] == 'ATF':
        return 'Antarctica'
    elif row['ISO 3166-1 alpha-3'] == 'TZA':
        return 'Tanzania'
    elif row['ISO 3166-1 alpha-3'] == 'BIH':
        return 'Bosnia & Herzegovina'
    elif row['ISO 3166-1 alpha-3'] == 'PRK':
        return 'Korea, South'
    elif row['ISO 3166-1 alpha-3'] == 'SGS':
        return 'sgs'
    return row['Country']

In [24]:
#remove space after name
df_meteorites.rename(columns={"English short name lower case": 'Country'}, inplace=True)
df_meteorites['Country'] = df_meteorites.apply(lambda row : rename_country(row), axis=1)
df_meteorites = df_meteorites.merge(df_countries, on=['Country'])
df_meteorites

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,ISO 3166-1 alpha-3,Country,Region,Population,Area (km sq),Pop. Density (per sq. km.),GDP ($ per capita),Climate
0,Aachen,1,Valid,L5,21.0,Fell,1880,50.77500,6.08333,DEU,Germany,WESTERN EUROPE,82422299,357021,"230,9",27600.0,3
1,Barntrup,4948,Valid,LL4,17.0,Fell,1886,52.00000,9.10000,DEU,Germany,WESTERN EUROPE,82422299,357021,"230,9",27600.0,3
2,Breitscheid,5134,Valid,H5,1500.0,Fell,1956,50.66694,8.18361,DEU,Germany,WESTERN EUROPE,82422299,357021,"230,9",27600.0,3
3,Bremervörde,5135,Valid,H/L3.9,7250.0,Fell,1855,53.40000,9.10000,DEU,Germany,WESTERN EUROPE,82422299,357021,"230,9",27600.0,3
4,Darmstadt,6603,Valid,H5,100.0,Fell,1804,49.86667,8.65000,DEU,Germany,WESTERN EUROPE,82422299,357021,"230,9",27600.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31906,Rosario,22765,Valid,"Iron, IAB-MG",2700.0,Found,1896,14.60000,-88.68333,HND,Honduras,LATIN AMER. & CARIB,7326496,112090,"65,4",2600.0,2
31907,Ryder Gletcher,22790,Valid,L5,5288.0,Found,1988,81.16667,-49.00000,GRL,Greenland,NORTHERN AMERICA,56361,2166086,"0,028",20000.0,1
31908,Sainte Rose,23720,Valid,H3.6,430.0,Found,1983,-21.19222,55.68639,REU,Reunion,SUB-SAHARAN AFRICA,787584,2517,"312,9",5800.0,2
31909,Shohaku,23580,Valid,Iron,101.0,Found,1938,40.31667,126.91667,PRK,"Korea, South",ASIA (EX. NEAR EAST),48846823,98480,496,17800.0,3


In [25]:
def rename_ISO(row):
    if row['ISO 3166-1 alpha-3'] == 'ATF':
        return 'ATA'
    return row['ISO 3166-1 alpha-3']

df_meteorites['ISO 3166-1 alpha-3'] = df_meteorites.apply(lambda row : rename_ISO(row), axis=1)

# Read previous csv to change climate to categorical

In [26]:
# df = pd.read_csv('./data/meteorites_country_V3.csv')
df = df_meteorites

def get_climate(row):
    if row['Climate'] == '1':
        return 'Desert'
    elif row['Climate'] == '2':
        return 'Tropical'
    elif row['Climate'] == '3':
        return 'Temperate'
    elif row['Climate'] == '4':
        return 'Temperate'
    elif row['Climate'] == '1,5':
        return 'Desert/Tropical'
    elif row['Climate'] == '2,5':
        return 'Tropical/Temperate'
    elif row['Country'] in ['Morocco', 'Libya']:
        return 'Desert'
    elif row['Country'] in ['Tanzania']:
        return 'Tropical'
    elif row['Country'] in ['Russia', 'Canada', 'Italy', 'Slovenia', 'Serbia']:
        return 'Temperate'
    else:
        return 'other'
    
df['Climate'] = df.apply(lambda row : get_climate(row), axis=1)
df['Climate'].unique()

array(['Temperate', 'Desert/Tropical', 'Desert', 'Tropical/Temperate',
       'Tropical', 'other'], dtype=object)

In [27]:
df.to_csv (r'./meteorites_country.csv', index = False, header=True)

# Data by country and climate
Aggregating data by courtry and then by climate

In [87]:
df = pd.read_csv('./meteorites_country.csv')
df_coordinates = pd.read_csv('./data/countries_coordinates.csv')

df_by_country = df.groupby(['Country', 'ISO 3166-1 alpha-3', 'Population', 'Area (km sq)', 'Pop. Density (per sq. km.)', 'GDP ($ per capita)', 'Climate', 'Region', 'fall'])[['fall']].count()

#df_by_country = df_by_country.reset_index(level=8)
df_by_country.rename(columns={"fall": 'count'}, inplace=True)
df_by_country = df_by_country.reset_index()
df_by_country.columns
# df_by_country.rename(columns={"name": "count"}, inplace=True)
# df_by_country.sort_values(by='count', inplace=True)



Index(['Country', 'ISO 3166-1 alpha-3', 'Population', 'Area (km sq)',
       'Pop. Density (per sq. km.)', 'GDP ($ per capita)', 'Climate', 'Region',
       'fall', 'count'],
      dtype='object')

In [85]:
df_coordinates.rename(columns={"Alpha-3 code": 'ISO 3166-1 alpha-3'}, inplace=True)
df_by_country = df_by_country.merge(df_coordinates, on=['ISO 3166-1 alpha-3'], how='left')

df_by_country
# df_by_country[df_by_country['Latitude'].isna()]
df_by_country.to_csv (r'./by_country.csv', index = False, header=True)

In [86]:
df_climate = df_by_country.groupby(['Climate'])[['count', 'Area (km sq)']].sum().reset_index()
df_climate.rename(columns={"name": "count"}, inplace=True)
df_climate.sort_values(by='count', inplace=True)
df_climate['density'] = df_climate['count']/df_climate['Area (km sq)']
df_climate.to_csv (r'./by_climate.csv', index = False, header=True)